In [65]:
import tiktoken
from openai import OpenAI

tokenizer = tiktoken.get_encoding("cl100k_base")
client = OpenAI()

In [66]:
MODEL = "gpt-4o"
RESPONSE_MODEL = "gpt-4o-mini"

In [67]:
system_message = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent $15. She has $8 left."""

system_message_tokens = len(tokenizer.encode(system_message))
print("Tokens:", system_message_tokens)
print()
print(system_message)

Tokens: 710

Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Ja

In [68]:
example_question = (
    "When I was 6 my sister was half my age. Now I’m 70 how old is my sister?"
)
example_responses = [
    "When I was 6 my sister was half my age, so she was 3. Now I am 70, so she is 70 - 3 = 67. The answer is 67.",
    "When the narrator was 6, his sister was half his age, which is 3. Now that the narrator is 70, his sister would be 70 - 3 = 67 years old. The answer is 67.",
    "When I was 6 my sister was half my age, so she was 3. Now I am 70, so she is 70/2 = 35. The answer is 35.",
]

In [69]:
def user_prompt(question: str) -> str:
    return f"Q: {question}\nA: "


print(user_prompt(example_question))

Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A: 


In [70]:
def response_prompt(responses: list[str]) -> str:
    solving_examples = "\n\n".join(
        [
            f"""Response {i}:
```
{example}
```"""
            for i, example in enumerate(responses)
        ]
    )
    return f"""{solving_examples}

Write an answer using the most commonly appearing response.
    """

In [71]:
print(response_prompt(example_responses))

Response 0:
```
When I was 6 my sister was half my age, so she was 3. Now I am 70, so she is 70 - 3 = 67. The answer is 67.
```

Response 1:
```
When the narrator was 6, his sister was half his age, which is 3. Now that the narrator is 70, his sister would be 70 - 3 = 67 years old. The answer is 67.
```

Response 2:
```
When I was 6 my sister was half my age, so she was 3. Now I am 70, so she is 70/2 = 35. The answer is 35.
```

Write an answer using the most commonly appearing response.
    


In [72]:
def ask(question: str, n=3, debug=False) -> str:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": user_prompt(question),
            },
        ],
        n=3,
    )

    responses = [choice.message.content for choice in response.choices]

    if debug:
        for i, choice in enumerate(responses):
            print(f"Response {i+1}:")
            print(choice)

    response = client.chat.completions.create(
        model=RESPONSE_MODEL,
        messages=[
            {
                "role": "user",
                "content": response_prompt(responses),
            },
        ],
    )

    answer = response.choices[0].message.content
    if debug:
        print("Answer:")
        print(answer)

    return answer

In [73]:
answer = ask(example_question, 3, True)

Response 1:
When you were 6 years old, your sister was half your age, which means she was \( \frac{6}{2} = 3 \) years old. The difference in age between you and your sister is 6 - 3 = 3 years. 

Now that you are 70 years old, your sister is 70 - 3 = 67 years old. 

The answer is 67.
Response 2:
When you were 6, your sister was half your age, which means she was \( \frac{1}{2} \times 6 = 3 \) years old. The age difference between you and your sister is \( 6 - 3 = 3 \) years.

Since this age difference does not change, now that you are 70, your sister is \( 70 - 3 = 67 \) years old. The answer is 67.
Response 3:
When you were 6, your sister was half your age, meaning she was \( \frac{6}{2} = 3 \) years old. The difference in age between you and your sister is \( 6 - 3 = 3 \) years.

Now that you are 70 years old, your sister is still 3 years younger than you. Therefore, she is \( 70 - 3 = 67 \) years old. The answer is 67.
Answer:
When you were 6 years old, your sister was half your age,